In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm 
from time import sleep

**собираем названия всех региональных центров с 2-ух источников**

In [2]:
# собираем названия регионалльных центров с 1-ого источника: https://marketing-course.ru/administrativnye-centry-regionov/
regional_centers_1 = set() # множество, т.к. у некоторых различных регионов одинаковые центры 
my_response = requests.get('https://marketing-course.ru/administrativnye-centry-regionov/').text
soup = BeautifulSoup(my_response, 'lxml')
table = soup.find_all('table')[0]
list_regions_info = table.find_all('tr')[1:]
for region in list_regions_info:
    title_regional_center = region.find_all('td')[3].text
    regional_centers_1.add(title_regional_center)
print(regional_centers_1, end="\n\n")
print(len(regional_centers_1))

{'Ярославль', 'Владивосток', 'Кызыл', 'Барнаул', 'Рязань', 'Псков', 'Магадан', 'Мурманск', 'Чебоксары', 'Благовещенск', 'Кемерово', 'Курган', 'Тюмень', 'Санкт-Петербург', 'Краснодар', 'Астрахань', 'Иркутск', 'Уфа', 'Владивкавказ', 'Магас', 'Владимир', 'Кострома', 'Петропавловск-Камчатский', 'Сыктывкар', 'Ижевск', 'Горно-Алтайск', 'Липецк', 'Махачкала', 'Ульяновск', 'Челябиннск', 'Улан-Удэ', 'Самара', 'Томск', 'Йошкар-Ола', 'Абакан', 'Ханты-мансийск', 'Хабаровск', 'Оренбург', 'Москва', 'Великий Новгород', 'Волгоград', 'Анадырь', 'Калуга', 'Саранск', 'Пермь', 'Якутск', 'Салехард', 'Курск', 'Калининград', 'Петрозаводск', 'Южно-Сахалинск', 'Нальчик', 'Ставрополь', 'Майкоп', 'Омск', 'Тамбов', 'Екатеринбург', 'Пенза', 'Черкесск', 'Брянск', 'Нижний Новгород', 'Казань', 'Ростов', 'Саратов', 'Воронеж', 'Иваново', 'Киров', 'Элиста', 'Орел', 'Тверь', 'Белгород', 'Грозный', 'Архангельск', 'Вологда', 'Новосибирск', 'Чита', 'Смоленск', 'Биробиджан', 'Тула', 'Красноярск'}

80


In [3]:
# собираем названия регионалльных центров со 2-ого источника: https://se33.ru/2-uncategorised/378-regions.html
regional_centers_2 = set() 
my_response = requests.get('https://se33.ru/2-uncategorised/378-regions.html').text
soup = BeautifulSoup(my_response, 'lxml')
table = soup.find('table')
list_regions_info = table.find_all('tr')[1:]
for region in list_regions_info:
    title_regional_center = region.find_all('td')[3].text
    regional_centers_2.add(title_regional_center)
print(regional_centers_2, end="\n\n")
print(len(regional_centers_2))

{'пгт Палана', 'Ярославль', 'Владивосток', 'Кызыл', 'Барнаул', 'Ростов-на-Дону', 'Рязань', 'Псков', 'Магадан', 'Мурманск', 'Чебоксары', 'Благовещенск', 'Нарьян-Мар', 'Кемерово', 'Курган', 'Тюмень', 'Санкт-Петербург', 'Краснодар', 'Астрахань', 'Иркутск', 'Уфа', 'Владимир', 'Кострома', 'Петропавловск-Камчатский', 'Сыктывкар', 'Ижевск', 'Челябинск', 'Горно-Алтайск', 'пгт Агинское', 'Липецк', 'Махачкала', 'пгт Тура', 'Ульяновск', 'Улан-Удэ', 'Самара', 'Назрань', 'Томск', 'Йошкар-Ола', 'Абакан', 'Хабаровск', 'Оренбург', 'Москва', 'Волгоград', 'Анадырь', 'Ханты-Мансийск', 'Калуга', 'Саранск', 'Пермь', 'Якутск', 'Дудинка', 'Салехард', 'Курск', 'Калининград', 'Южно-Сахалинск', 'Петрозаводск', 'Нальчик', 'Ставрополь', 'Майкоп', 'Омск', 'Тамбов', 'Екатеринбург', 'Пенза', 'Черкесск', 'Брянск', 'Нижний Новгород', 'Казань', 'пгт Усть-Ордынский', 'Саратов', 'Воронеж', 'Иваново', 'Киров', 'Элиста', 'Владикавказ', 'Орел', 'Тверь', 'Белгород', 'Грозный', 'Кудымкар', 'Архангельск', 'Новгород', 'Вологда'

**объединяем оба множества (union)**

In [4]:
all_regional_centers = regional_centers_1.union(regional_centers_2)
print(all_regional_centers, end='\n\n')
print(len(all_regional_centers))

{'пгт Палана', 'Ярославль', 'Владивосток', 'Кызыл', 'Барнаул', 'Рязань', 'Ростов-на-Дону', 'Псков', 'Магадан', 'Мурманск', 'Чебоксары', 'Благовещенск', 'Нарьян-Мар', 'Кемерово', 'Курган', 'Тюмень', 'Санкт-Петербург', 'Краснодар', 'Астрахань', 'Иркутск', 'Уфа', 'Владивкавказ', 'Магас', 'Владимир', 'Кострома', 'Петропавловск-Камчатский', 'Сыктывкар', 'Ижевск', 'Челябинск', 'Горно-Алтайск', 'пгт Агинское', 'Липецк', 'Махачкала', 'пгт Тура', 'Ульяновск', 'Челябиннск', 'Улан-Удэ', 'Самара', 'Назрань', 'Томск', 'Йошкар-Ола', 'Абакан', 'Ханты-мансийск', 'Хабаровск', 'Оренбург', 'Москва', 'Великий Новгород', 'Волгоград', 'Анадырь', 'Ханты-Мансийск', 'Калуга', 'Саранск', 'Пермь', 'Якутск', 'Дудинка', 'Салехард', 'Курск', 'Калининград', 'Петрозаводск', 'Южно-Сахалинск', 'Нальчик', 'Ставрополь', 'Майкоп', 'Омск', 'Тамбов', 'Екатеринбург', 'Пенза', 'Черкесск', 'Брянск', 'Нижний Новгород', 'Казань', 'пгт Усть-Ордынский', 'Ростов', 'Саратов', 'Воронеж', 'Иваново', 'Киров', 'Элиста', 'Владикавказ', '

In [5]:
titles = []
adresses = []
websites = []
phone_numbers = []
emails = []

In [6]:
def input_info(): # поиск информации
    input_tab = browser.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[2]/div[2]/div[2]/div[2]/div/div/input")
    input_tab.send_keys(regional_center)
    
def click_search_button(): # нажатие на кнопку поиска
    search_button = browser.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[2]/div[2]/div[2]/div[2]/div/div/button[2]")
    search_button.click()

def click_clear_info_button(): # нажатие на кнопку удаления текста в области поиска
    clear_info_button = browser.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[2]/div[2]/div[2]/div[2]/div/div/button[1]')
    clear_info_button.click()
    
    
def collect_data(): # сбор всех данных(адрес, сайт, телефон, почта) с найденного представителя поддержки бизнеса 
    try:
        title = browser.find_element(By.XPATH, '//*[@id="map"]/div[1]/div[6]/div/div/div/div/div/div/div').text
        adress = browser.find_element(By.XPATH, '//*[@id="map"]/div[1]/div[6]/div/div/div/div/div/div/table/tbody/tr[1]/td[2]').text
        website = browser.find_element(By.XPATH, '//*[@id="map"]/div[1]/div[6]/div/div/div/div/div/div/table/tbody/tr[2]/td[2]/a').get_attribute('href')
        phone = browser.find_element(By.XPATH, '//*[@id="map"]/div[1]/div[6]/div/div/div/div/div/div/table/tbody/tr[3]/td[2]').text
        email = browser.find_element(By.XPATH, '//*[@id="map"]/div[1]/div[6]/div/div/div/div/div/div/table/tbody/tr[4]/td[2]/a').text
    except NoSuchElementException:
        title = None
        adress = None
        website = None
        phone = None
        email = None 
    titles.append(title)
    adresses.append(adress)
    websites.append(website)
    phone_numbers.append(phone)
    emails.append(email)

In [7]:
browser = Chrome()
url = 'https://xn--l1agf.xn--p1ai/services/reg-support-map/'
browser.maximize_window() 
browser.get(url)
sleep(1) # пауза, чтобы страницы успела прогрузиться 
for regional_center in all_regional_centers:
    input_info()
    click_search_button()
    sleep(0.8)
    collect_data()
    click_clear_info_button()

In [8]:
data = {'название': titles, 'адрес': adresses, 'сайт': websites, 'телефон': phone_numbers, 'почта': emails}
df = pd.DataFrame(data=data)

In [9]:
df

,название,адрес,сайт,телефон,почта
0,None,None,None,None,None
1,"ГБУ ЯО ""Корпорация развития МСП""","Ярославская обл, г Ярославль, ул Свердлова, 25...",http://corpmsp76.ru/,+7 (4852) 370-401,Info@corpmsp76.ru
2,АНО «Центр поддержки предпринимательства Примо...,"Приморский край, г Владивосток, ул Тигровая, 7...",http://xn--25-9kcqjffxnf3b.xn--p1ai/,+7800 555 09 33,office@cpp25.ru
3,МКК Фонд ПП РТ,"Тыва, Кызыл, Тувинских Добровольцев, 18, 213, ...",http://xn--17-9kcqjffxnf3b.xn--p1ai/,+7 39422-36202,bsf-tuva@mail.ru
4,"НО ""Алтайский фонд МСП""","Алтайский, Барнаул, Мало-Тобольская, 19, 656056",http://xn--22-9kcqjffxnf3b.xn--p1ai/,+7 3852 57-34-50,info@altfond.ru
...,...,...,...,...,...
88,МКК ФПМП ЗАБАЙКАЛЬСКОГО КРАЯ (ФОНД),"Забайкальский край, г Чита, ул Бабушкина, 52, ...",http://xn--75-9kcqjffxnf3b.xn--p1ai/,+7 800 100-10-22,mailbox@zabbusiness.ru
89,"АНО ""ЦПП Смоленской области""","Смоленская область, Смоленск, Тенишевой, 15, 1...",http://cpp67.ru/,+7 481 263-80-38 доб. 6,info@cpp67.ru
90,"НКО ФОНД ""ИНВЕСТИЦИОННОЕ АГЕНТСТВО ЕАО""","Еврейская, Биробиджан, 60-летия СССР, 12А, 679016",http://mybusiness79.ru/,8 800 550-85-79,apieao@post.eao.ru
91,ТРФ ЦПП,"Тульская, Тула, Кирова, 135, 1, 300004",https://xn--80abmheescnf3bmn.xn--p1ai/,8-800-600-77-71,info@mb71.ru


**добавляем в df столбцы, в которые отделно выделяем регионы, города и улицы для возможности быстрой фильтрации**

In [10]:
postal_codes = []
regions = []
cities = []
streets = []

def add_information_to_lists(number_postal_code, name_region, name_sity, name_street):
    postal_codes.append(number_postal_code)
    regions.append(name_region)
    cities.append(name_sity)
    streets.append(name_street)

def get_postal_code(full_adress):
    return full_adress.split(',')[-1]

def get_region(full_adress):
    return full_adress.split(',')[0]

def get_city(full_adress):
    return full_adress.split(',')[1]

def get_street(full_adress):
    return full_adress.split(',')[2]

for full_adress in df['адрес']:
    if full_adress != None:
        postal_code = get_postal_code(full_adress)
        region = get_region(full_adress)
        city = get_city(full_adress)
        street = get_street(full_adress)
    else:
        postal_code = region = city = street = None 
    add_information_to_lists(postal_code, region, city, street)

In [11]:
df['регион'] = regions
df['город'] = cities
df['улица'] = streets
df['почтовый индекс'] = postal_codes

**удаляем мусор в виде приставок "г", "ул" и тд, а также пустые строки**

In [12]:
df = df.dropna(subset=['название'])

In [13]:
df['регион'] = df['регион'].apply(lambda region_name: region_name.replace('г', '').strip())
df['город'] = df['город'].apply(lambda city_name: city_name.replace('г ', '').replace('г. ', '').replace('город', '').strip())
df['улица'] = df['улица'].apply(lambda street_name: street_name.replace('ул ', '').replace('ул. ', '').replace('пр-кт', '').replace('проспект', '').strip())

C:\Users\1\AppData\Local\Temp\ipykernel_15624\3185225848.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['регион'] = df['регион'].apply(lambda region_name: region_name.replace('г', '').strip())
C:\Users\1\AppData\Local\Temp\ipykernel_15624\3185225848.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['город'] = df['город'].apply(lambda city_name: city_name.replace('г ', '').replace('г. ', '').replace('город', '').strip())
C:\Users\1\AppData\Local\Temp\ipykernel_15624\3185225848.py:3: SettingWithC

In [14]:
df.to_excel("list_business_support_representatives.xlsx", index=False)
df.head()

,название,адрес,сайт,телефон,почта,регион,город,улица,почтовый индекс
1,"ГБУ ЯО ""Корпорация развития МСП""","Ярославская обл, г Ярославль, ул Свердлова, 25...",http://corpmsp76.ru/,+7 (4852) 370-401,Info@corpmsp76.ru,Ярославская обл,Ярославль,Свердлова,150000
2,АНО «Центр поддержки предпринимательства Примо...,"Приморский край, г Владивосток, ул Тигровая, 7...",http://xn--25-9kcqjffxnf3b.xn--p1ai/,+7800 555 09 33,office@cpp25.ru,Приморский край,Владивосток,Тигровая,690091
3,МКК Фонд ПП РТ,"Тыва, Кызыл, Тувинских Добровольцев, 18, 213, ...",http://xn--17-9kcqjffxnf3b.xn--p1ai/,+7 39422-36202,bsf-tuva@mail.ru,Тыва,Кызыл,Тувинских Добровольцев,667000
4,"НО ""Алтайский фонд МСП""","Алтайский, Барнаул, Мало-Тобольская, 19, 656056",http://xn--22-9kcqjffxnf3b.xn--p1ai/,+7 3852 57-34-50,info@altfond.ru,Алтайский,Барнаул,Мало-Тобольская,656056
5,АНО,"Рязанская обл, г Рязань, ул Каширина, 1Б, 2 по...",http://agency62.ru/,+7 491 260-60-00,ano.arb@yandex.ru,Рязанская обл,Рязань,Каширина,390000
